# Generative AI for Audio Application

#### Restart Kernel (If needed)

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# Import Libraries
Most of the complexity for the chatbot is in [customizable_chatbot.py](./customizable_chatbot.py) that uses [audio.py](./audio.py) internally for the audio capabilities.

In [ ]:
import sys
print(sys.path[-1])

In [ ]:
import os
import gradio as gr
from genai_voice.bots.chatbot import ChatBot
from genai_voice.config.defaults import Config
from genai_voice.logger.log_utils import log, LogLevels

In [ ]:
log(f'Configuration: {Config()}', log_level=LogLevels.ON)

## Set Current Working Directory  

We want to simulate running this notebook from the project root just as it would work when using `Poetry` scripts.

In [ ]:
curr_wrk_dir = os.getcwd()
log(f'Before directory change: {curr_wrk_dir}')
if curr_wrk_dir.endswith('app'):
    log(f'Changing directory to root')
    os.chdir('..')
    curr_wrk_dir = os.getcwd()
log(f'Before directory change: {curr_wrk_dir}')

# Create Data for LLM Context  

`Poetry` scripts allow us to install our code as a package and run functions executables. 

We will use the `ExtractWebPagesAndSaveData` script, that is defined in `pyproject.toml` to scrape, extract and generate the file that the LLM will use as context data.

`SAMPLE_URLS` have been defined under provided under `genai_voice.data_utils.urls.py`. Feel free to modify the links in that file to customize the source of data. 

> **DISCLAIMER:** Be responsible when scraping data that is not yours, complying with the EULA of the sites and conducting it in a legal fashion. Also remember that most sites will throttle scrapes, so do this with caution.  

> **NOTE:** This is an optional step. It just shows you have you can get custom data for your LLM context. We have provided the data for this project. 

In [ ]:
!poetry run ExtractWebPagesAndSaveData

## Gradio Interface
This launches the UI, you will probably need to allow the browser to use the microphone to enable the audio functions.

In [ ]:
"""Run Chatbot app"""
chatbot = ChatBot(enable_speakers=True, threaded=True)
history = []

def get_response(audio):
    """Get Audio Response From Chatbot"""
    if not audio:
        raise ValueError("No audio file provided.")
    prompt = chatbot.get_prompt_from_gradio_audio(audio)
    log(f"Transcribed prompt: {prompt}", log_level=LogLevels.ON)
    response = chatbot.respond(prompt, history)
    log(f"Chatbot response: {response}", log_level=LogLevels.ON)
    history.append([prompt, response])
    return response

demo = gr.Interface(
        get_response,
        gr.Audio(sources="microphone"),
        None,
        title="Wanderwise Travel Assistant"
)
demo.launch()